# Transfer learning using Keras

## Imports and constants

In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

import owkin.utils as utils
from pathlib import Path
import logging
import numpy as np
import pandas as pd

utils.configure_logger()

/Users/jeremy.guillon/Repositories/owkin-breast-cancer-challenge


In [50]:
# data properties:
HEIGHT = 224 # our images height (in pixels)
WIDTH = 224 # our images width (in pixels)
CHANNELS = 3 # our images channels (i.e. RVB)
DATA_DIR = 'data' # data directory where .csv files are stored
IMAGES_DIR = 'data/train_input/images' # directory where images are stored
BEST_MODEL_FILENAME = 'models/best_transfer_learning.hd5' # output model filename

# deep neural network architecture:
HIDDEN_LAYER_SIZE = 32 # number of units in the last dense layer
OUTPUT_LAYER_SIZE = 2 # number of classes to predict (here "non-tumoral" and "tumoral")
RESNET_POOLING = 'avg' # ['avg' | 'max' ]

# training procedure hyperparameters:
VALIDATION_SPLIT = 0.2 # proportion of images to be reserved for the validation dataset
BATCH_SIZE = 8
NUM_EPOCHS = 20
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10
LOSS_FUNCTION = 'categorical_crossentropy'
METRICS = ['accuracy']

# optimizer hyperparameters:
LEARNING_RATE = 0.001
DECAY = 1e-6
MOMENTUM = 0.9

# others:
SEED = 42 # seed number for reproducibility of the training validation splits

## Model definition

We want to use a pretrained version of ResNet50, on the ImageNet dataset, and fine-tune it in order to classify tumoral from non-tumoral tiles. To do so, we add fully-connected (FC or Dense) layers and train them on our annotated dataset.

In [44]:
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense

def build_model():
    model = Sequential([
        ResNet50(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, CHANNELS), pooling=RESNET_POOLING),
        Dense(HIDDEN_LAYER_SIZE, activation='relu'),
        Dense(OUTPUT_LAYER_SIZE, activation='softmax')
    ])
    
    # we do not train the ResNet50 layer since it is already pre-trained on the ImageNet dataset
    model.layers[0].trainable = False
    
    return model

model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 32)                65568     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 23,653,346
Trainable params: 65,634
Non-trainable params: 23,587,712
_________________________________________________________________


## Image generator

Since our dataset contains "only" arround 10k images, we might need to *augment* it, that's **data augmentation**. It consists, in our case, in rotating or flipping the tiles to make as if they were different images.

In [22]:
annot_tiles_filename = DATA_DIR + "/train_input" + "/train_tile_annotations.csv"
logging.debug(f'reading csv file: {annot_tiles_filename}')
annot_tiles_df = pd.read_csv(annot_tiles_filename)

reading csv file: data/train_input/train_tile_annotations.csv


Let's visualize the `csv` file content:

In [24]:
annot_tiles_df.head()

,Unnamed: 0,Target
0,ID_387_annotated_tile_0_15_69_30.jpg,0.0
1,ID_387_annotated_tile_1_15_23_53.jpg,0.0
2,ID_387_annotated_tile_2_15_58_20.jpg,0.0
3,ID_387_annotated_tile_3_15_67_12.jpg,0.0
4,ID_387_annotated_tile_4_15_57_20.jpg,0.0


We create an image generator that will load on-the-fly batches of images in memory; raw and/or augmented (i.e. artifical) ones. The dataset will be randomly split in training and validation sets according to the `VALIDATION_SPLIT` parameter.

In [32]:
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

data_generator =  ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=VALIDATION_SPLIT,
    rotation_range=90,
    horizontal_flip=True,
    vertical_flip=True
)

training_generator = data_generator.flow_from_dataframe(
    subset='training', # set it as the training dataset
    dataframe=annot_tiles_df, x_col='Unnamed: 0', y_col='Target',
    directory=IMAGES_DIR, # images directory where are stored files whose filenames are listed in the `x_col` of the dataframe
    seed=SEED,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary')

validation_generator = data_generator.flow_from_dataframe(
    subset='validation', # set it as the validation dataset
    dataframe=annot_tiles_df, x_col='Unnamed: 0', y_col='Target',
    directory=IMAGES_DIR, # images directory where are stored files whose filenames are listed in the `x_col` of the dataframe
    seed=SEED,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary')

Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


## Model training

### Optimizer and callbacks

In [54]:
from keras.optimizers import SGD, Adam

sgd = SGD(lr=LEARNING_RATE, decay=DECAY, momentum=MOMENTUM, nesterov=True)
model.compile(optimizer=sgd, loss=LOSS_FUNCTION, metrics=METRICS)

from keras.callbacks import ModelCheckpoint, TensorBoard
checkpoint_cb = ModelCheckpoint(filepath=BEST_MODEL_FILENAME, 
                                monitor='val_loss', 
                                save_best_only=True)
tensorboard_cb = TensorBoard(log_dir='logs', 
                             batch_size=BATCH_SIZE, 
                             write_graph=True,
                             update_freq='batch')

### Training

In [46]:
model.fit_generator(
    training_generator,
    steps_per_epoch=STEPS_PER_EPOCH_TRAINING,
    epochs = NUM_EPOCHS,
    validation_data=validation_generator,
    validation_steps=STEPS_PER_EPOCH_VALIDATION,
    callbacks=[checkpoint_cb, tensorboard_cb]
)

Instructions for updating:
Use tf.cast instead.


Epoch 1/20


ZeroDivisionError: integer division or modulo by zero